# Bayesian Inference: Theory

In [1]:
# Import some helper functions (please ignore this!)
from utils import *

**Context:** 

**Challenge:**

**Outline:**

## The Need for Uncertainty Quantification

**The MLE is Over-Confident.** In safety-critical contexts, like those from the IHH, it's important that our ML models don't just fit the observed data well; they should also communicate with us the limits of their "knowledge." Let's illustrate what we mean. Consider the regression data below:

TODO: figure with in-between and OOD uncertainty. 

As you see in the figure above, we don't have data for some segments of the population of interest. 

TODO: explain why this is bad in the context of the task. 

TODO: give example with cats and COVID

Ideally, our learning algorithm would give us options; it would give us several models that all fit the data reasonably well, but behave differently away from the data. Given these options, perhaps we could devise some algorithm to select the one we would like to use on our downstream task, or find some way to *combine* them. Unfortunately, the learning algorithm we've use so far---the MLE---doesn't provide us with a way to do this. The MLE gives a *single* model. 

**Ensembling.** One way to solve this issue is by relying on the imperfections of the optimizer. Remember that, especially for more expressive models, optimization tends to get stuck in local optima. What if we were to collect an *ensemble* of models, all fit with the MLE to data, but each optimized from a different random initialization? Because each model would get stuck in a different local optima, each *might* behave differently than the others away from the data. What's nice about this approach is that it's easy to implement: we already have all the tools we need! Let's see what ensembling a neural network regression model looks like:

TODO figure of NN ensemble on above data

While effective in practice, ensembling also has one big problem when it comes to safety-critical contexts. It makes implicit assumptions that are difficult to understand. Specifically, we relied on the imperfections of our black-box optimizer to find us a diverse set of models. What kind of models will the optimizer give us, however? Do these models have an *inductive bias* that are appropriate for our task? 

The need for explicit assumptions motivates us to find an alternative way of fitting our models to day, leading us to the *Bayesian approach*. 

## Fitting Models via Bayes' Rule

**The Bayesian Paradigm.** So let's go back to the drawing board and rethink how we've been fitting models this whole time. So far, our approach has been finding the *single* model that maximizes the probability of our observed data: $\theta^\text{MLE} = \log p(\mathcal{D}; \theta)$. But isn't what we're actually interested is the *distribution* of models given the data, $p(\theta | \mathcal{D})$? In other words, conditioned on the data we've observed so far, we want to know which models (represented by their parameters, $\theta$) are likely to fit the data well. In this new paradigm, we hope that:
1. $p(\theta | \mathcal{D})$ will capture a diversity of models with different inductive biases.
2. $p(\theta | \mathcal{D})$ will make our assumptions clear, and will allow us to specify what type of inductive biases are appropriate for our task.

**Bayes' Rule.** But what is $p(\theta | \mathcal{D})$, exactly? How can we possibly write down a distribution of models that fit the data well by hand? Isn't the whole point that the machine will do the learning for us? To get around this problem, we will use *Baye's rule* to write down $p(\theta | \mathcal{D})$ in terms of what we already know how to specify: $p(\mathcal{D} | \theta)$.

Recall from the chapter on joint probability that a joint distribution over two random variables, $A$ and $B$, can be factorized as follows:
\begin{align}
p_{A, B}(a, b) &= p_{B | A}(b | a) \cdot p_A(a) \quad (\text{Option 1}) \\
&= p_{A | B}(a | b) \cdot p_B(b) \quad (\text{Option 2})
\end{align}
This means we can also equate the two factorizations:
\begin{align}
p_{B | A}(b | a) \cdot p_A(a) &= p_{A | B}(a | b) \cdot p_B(b)
\end{align}
Diving both sides by $p_A(a)$, we get:
\begin{align}
p_{B | A}(b | a) &= \frac{p_{A | B}(a | b) \cdot p_B(b)}{p_A(a)} \quad \text{(Bayes' Rule)}
\end{align}
This is Bayes' rule. What's cool about it is that relates $p_{B | A}(b | a)$ to $p_{A | B}(a | b)$. 

Putting this back in the context of our problem, let's treat $\mathcal{D}$ *and* $\theta$ as random variables. Applying Bayes' rule, we can relate $p(\theta | \mathcal{D})$ (which we don't know how to specify) to $p(\mathcal{D} | \theta)$ (which we do know how to specify):
\begin{align}
p(\theta | \mathcal{D}) &= \frac{p(\mathcal{D} | \theta) \cdot p(\theta)}{p(\mathcal{D})}
\end{align}

## Deriving the Predictive Distribution